In [918]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from math import *

In [919]:
frame = cv2.imread("capture_mire_0.png")

#plt.imshow(frame)
retval1, corners1 = cv2.findChessboardCorners(frame, (7,7))

frame1 = cv2.imread("capture_mire_1.png")

#plt.imshow(frame1)
retval2, corners2 = cv2.findChessboardCorners(frame1, (7,7))



In [920]:
coord_px1 = []
for i in range (0,len(corners1)) : 
    coord_px1.append(corners1[i,0].tolist())

coord_px2 = []
for i in range (0,len(corners2)) : 
    coord_px2.append(corners2[i,0].tolist())



In [921]:
coord_mm1 = np.zeros((49,3))
for i in range (0,49):
    x = i%7
    y = i//7
    coord_mm1[i] = [x * 20, y * 20, 0]

coord_mm2 = np.zeros((49,3))
for i in range (0,49):
    x = i%7
    y = i//7
    coord_mm2[i] = [x * 20, y * 20, 120]

In [922]:
A = np.zeros([98,7])
U1 = np.zeros([98,1])
i2 = frame.shape[0] / 2
i1 = frame.shape[1] / 2

K = np.zeros([98,7])
for i in range (0,49):
    for j in range (0,4):
        K[i][j] = coord_px1[i][1] - i2
for i in range (0,49):
    for j in range (4,7):
        K[i][j] = coord_px1[i][0] - i1

for i in range (49,98):
    for j in range (0,4):
        K[i][j] = coord_px2[i - 49][1] - i2
for i in range (49,98):
    for j in range (4,7):
        K[i][j] = coord_px2[i - 49][0] - i1

M = np.zeros([98,7])
for i in range (0,49):
    for j in range (0,3):
        M[i][j] = coord_mm1[i][j]
for i in range (0,49):
    M[i][3] = 1
for i in range (0,49):
    for j in range (4,7):
        M[i][j] = -1*coord_mm1[i][j - 4]
        
for i in range (49,98):
    for j in range (0,3):
        M[i][j] = coord_mm2[i - 49][j]
for i in range (49,98):
    M[i][3] = 1
for i in range (49,98):
    for j in range (4,7):
        M[i][j] = -1*coord_mm2[i - 49][j - 4]



for i in range (0,98):
    for j in range (0,7):
        A[i][j] = K[i][j] * M[i][j]

for i in range (0,49):
    U1[i] = coord_px1[i][0] - i1
for i in range (49,98):
    U1[i] = coord_px2[i - 49][0] - i1



In [923]:
o2 = 0
beta = 0
o1 = 0
r11,r12,r13,r21,r22,r23 = 0,0,0,0,0,0

Ainv= np.linalg.pinv(A)

L = np.dot(Ainv,U1)

o2 = 1/sqrt(L[4]**2 + L[5]**2 + L[6]**2)
beta = o2*sqrt(L[0]**2 + L[1]**2 + L[2]**2)
o2 = -o2
o1 = L[3]*o2/beta
r11 = L[0]*o2/beta
r12 = L[1]*o2/beta
r13 = L[2]*o2/beta
r21 = L[4]*o2
r22 = L[5]*o2
r23 = L[6]*o2

print("o1 = ",o1)
print("r11 = ",r11)
print("r12 = ",r12)
print("r13 = ",r13)
print("r21 = ",r21)
print("r22 = ",r22)
print("r23 = ",r23)

print("---------------------------")




o1 =  [-53.30894165]
r11 =  [0.98684761]
r12 =  [0.00086409]
r13 =  [-0.16165104]
r21 =  [0.17347513]
r22 =  [0.91753194]
r23 =  [0.3578289]
---------------------------


In [924]:
R1 = np.array([r11,r12,r13])
R2 = np.array([r21,r22,r23])

R3 = np.cross(R1.transpose(),R2.transpose())

r31 = R3[0][0]
r32 = R3[0][1]
r33 = R3[0][2]

phi = -atan(r23/r33)
gamma = -atan(r12/r11)
omega = atan(r13/(-r23*sin(phi)+r33*cos(phi)))


In [925]:
B =np.zeros([98,2])
R = np.zeros([98,1])
Z = np.zeros([98,3])
U2barre = np.zeros([98,1]) 

i2 = frame.shape[0] / 2
i1 = frame.shape[1] / 2

for i in range (0,49):
    U2barre[i] = coord_px1[i][1] - i2
    
i2 = frame1.shape[0] / 2
i1 = frame1.shape[1] / 2
for i in range (49,98):
    U2barre[i] = coord_px2[i-49][1] - i2

for i in range (0,49):
    for j in range (0,3):
        Z[i][j] = coord_mm1[i][j]
for i in range (49,98):
    for j in range (0,3):
        Z[i][j] = coord_mm2[i-49][j]

Y = np.dot(Z,R3.transpose())

for i in range (0,98):
    R[i]=-U2barre[i]*Y[i]
    B[i][0] = U2barre[i]

Y = np.dot(Z,R2)

for i in range (0,98):
    B[i][1] = -(Y[i] + o2)



In [926]:
Binv= np.linalg.pinv(B)

V = np.dot(Binv,R)
o3 = V[0]
f2 = V[1]
f1 = beta * f2
s1 = 4/f1
s2 = 4/f2



In [927]:
Mint = np.zeros([3,4])
Mext = np.zeros([4,4])

i2 = frame.shape[0] / 2
i1 = frame.shape[1] / 2

Mint[0][0] = 4/s1
Mint[1][1] = 4/s2
Mint[2][2] = 1
Mint[1][2] = i2
Mint[0][2] = i1

Mext[0][0] = r11
Mext[0][1] = r12
Mext[0][2] = r13
Mext[0][3] = o1

Mext[1][0] = r21
Mext[1][1] = r22
Mext[1][2] = r23
Mext[1][3] = o2

Mext[2][0] = r31
Mext[2][1] = r32
Mext[2][2] = r33
Mext[2][3] = o3

Mext[3][3] = 1

Id = np.ones([1,49])
new_coord= np.concatenate((coord_mm1,Id.transpose()), axis = 1)
U = np.dot(Mint,Mext)
U = np.dot(U, new_coord.transpose())

alpha = U[2][0]


In [928]:
for i in range(0,len(U[0])):
    print(i)
    U[0][i] = int(U[0][i]/U[2][i])
    U[1][i] = int(U[1][i]/U[2][i])

    print(" U[0][i] = ", U[0][i])
    print(" U[1][i] = ", U[1][i])
    print("---------------")
    frame[int(U[1][i]),int(U[0][i])] = [0,0,255]
    frame[int(U[1][i])+1,int(U[0][i])] = [0,0,255]
    frame[int(U[1][i]),int(U[0][i])+1] = [0,0,255]
    frame[int(U[1][i])+1,int(U[0][i])+1] = [0,0,255]
    frame[int(U[1][i]),int(U[0][i])-1] = [0,0,255]
    frame[int(U[1][i])-1,int(U[0][i])] = [0,0,255]
    frame[int(U[1][i]),int(U[0][i])-1] = [0,0,255]
    frame[int(U[1][i])+1,int(U[0][i])-1] = [0,0,255]
    frame[int(U[1][i])-1,int(U[0][i])+1] = [0,0,255]



cv2.imwrite("result1.png", frame)


0
 U[0][i] =  228.0
 U[1][i] =  97.0
---------------
1
 U[0][i] =  262.0
 U[1][i] =  106.0
---------------
2
 U[0][i] =  296.0
 U[1][i] =  114.0
---------------
3
 U[0][i] =  329.0
 U[1][i] =  122.0
---------------
4
 U[0][i] =  362.0
 U[1][i] =  130.0
---------------
5
 U[0][i] =  394.0
 U[1][i] =  138.0
---------------
6
 U[0][i] =  426.0
 U[1][i] =  145.0
---------------
7
 U[0][i] =  226.0
 U[1][i] =  133.0
---------------
8
 U[0][i] =  261.0
 U[1][i] =  141.0
---------------
9
 U[0][i] =  296.0
 U[1][i] =  149.0
---------------
10
 U[0][i] =  330.0
 U[1][i] =  157.0
---------------
11
 U[0][i] =  363.0
 U[1][i] =  165.0
---------------
12
 U[0][i] =  396.0
 U[1][i] =  173.0
---------------
13
 U[0][i] =  428.0
 U[1][i] =  180.0
---------------
14
 U[0][i] =  223.0
 U[1][i] =  170.0
---------------
15
 U[0][i] =  260.0
 U[1][i] =  178.0
---------------
16
 U[0][i] =  295.0
 U[1][i] =  186.0
---------------
17
 U[0][i] =  330.0
 U[1][i] =  194.0
---------------
18
 U[0][i] =  364.0


True